In [1]:
pwd

'/home/rnshishir/deepmed/CPRD_Cut22/debug'

In [2]:
import pyspark

In [3]:
import sys 
sys.path.insert(0, '/home/rnshishir/deepmed/CPRD_Cut22/')
import shutil
from utils.yaml_act import yaml_load
from utils.arg_parse import arg_paser
from CPRD.config.spark import spark_init, read_parquet
import pyspark.sql.functions as F
from CPRD.functions import tables, merge
from CPRD.functions import merge
from utils.utils import save_obj
from CPRD.functions.MedicalDictionary import * 
from CPRD.functions.Prediction import * 
from CPRD.functions.cohort_select_causal import * 
from CPRD.functions.predictor_extractor import *
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [4]:

def vocabCreate(CountsICD, num):
    dic={}
    dic['token2idx']={}
    dic['idx2token']={}
    dic['token2idx']['MASK']=4
    dic['token2idx']['CLS']=3
    dic['token2idx']['SEP']=2
    dic['token2idx']['UNK']=1
    dic['token2idx']['PAD']=0
    dic['idx2token'][4]='MASK'
    dic['idx2token'][3]='CLS'
    dic['idx2token'][2]='SEP'
    dic['idx2token'][1]='UNK'
    dic['idx2token'][0]='PAD'
    i=5
    for x in CountsICD:
        if CountsICD[x]>num:
            dic['token2idx'][x]=i
            dic['idx2token'][i]=x
            i=i+1
    return dic

In [5]:
args = dotdict({'params': '/home/rnshishir/deepmed/CPRD_Cut22/config/config.yaml'})
params = yaml_load(args.params)
spark_params = params['pyspark']
spark = spark_init(spark_params)
file = params['file_path']
data_params = params['params']

/home/rnshishir/deepmed/CPRD_Cut22/utils/yaml_act.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [6]:
# params have each of the individual files for data processing and other things like location of medical dict etc
params

{'task': 'medication.main',
 'pyspark': {'temp': '/home/rnshishir/sparkNewT',
  'pyspark_env': '/home/rnshishir/anaconda3/envs/pyspark37/bin/python3.7'},
 'file_path': {'clinical': '/home/workspace/datasets/cprd/cprd2021/*/*_Observation_*',
  'patient': '/home/workspace/datasets/cprd/cprd2021/*/*_Patient_*',
  'problem': '/home/workspace/datasets/cprd/cprd2021/*/*_Problem_*',
  'therapy': '/home/workspace/datasets/cprd/cprd2021/*/*_DrugIssue_*',
  'practice': '/home/workspace/datasets/cprd/cprd2021/*/*_Practice_*',
  'consultation': '/home/workspace/datasets/cprd/cprd2021/*/*_Consultation_*',
  'staff': '/home/workspace/datasets/cprd/cprd2021/*/*_Staff*',
  'diagnosis_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_diagnosis_hosp_20_095_DM.txt',
  'proc_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_procedures_epi_20_095_DM.txt',
  'op_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_09

# cohort selection
### all eligible patients between 1985 and 2020 (not up to for covid reasons) 

In [ ]:
# CohortSoftCut from the causal cohort selection package has everything to select cohort and baseline
# specifically the baseline for those WITH the exposure is date of exposure, and for those WITHOUT exp of interest is random sampling of baseline 
cohortSelector = CohortSoftCut(1,18, 200,  None, False, False, True)



In [ ]:
cohort = cohortSelector.demoExtract(file, spark, duration=('1985-01-01', '2020-01-01'))

cohort.write.parquet('/home/shared/shishir/AurumOut/rawDat/allelig_cohort_association_1985_2020.parquet')


In [18]:
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/allelig_cohort_association_1985_2020.parquet')
cohort.count()
# 259345 pats 

26437981

In [19]:
# cohort = cohort .sample(0.5)
# cohort.count()

13214057

In [20]:
cohort = cohort.withColumn('enddate_real', F.least(F.col('exit_date'), F.col('enddate'))).drop('enddate_real').withColumnRenamed('enddate_real', 'enddate')


In [21]:
cohort = cohort.withColumn('label', F.lit(1))
# this is just an include flag so we can include all elig patients (might be useful for future work hence putting it in now)



In [22]:
cohort=cohort.select([
    
    'patid',
    'gender',
    'region',
    'startdate',
    'enddate',
    'label',
    'dob'
])

In [23]:
allDiag = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')
meds = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/medications_1985_2022_bnfvtm.parquet')

allDiag = allDiag.select('patid','eventdate','ICD').withColumnRenamed('ICD','code')
meds = meds.select('patid','eventdate','code')
fullcodes = allDiag.union(meds)


### unique per year (i.e., delete duplicate rec in a given year)

In [24]:
extractorMLM = BEHRTextraction()
dataout = extractorMLM.format_behrt(fullcodes, cohort, col_entry = 'enddate', unique_in_months=6)

In [25]:
dataout.write.parquet('/home/shared/shishir/AurumOut/rawDat/MLM_for_pretraining_28M_1985_2020__unique_per6m_50pc_sample.parquet')


In [30]:
dat = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/MLM_for_pretraining_28M_1985_2020__unique_per6m_50pc_sample.parquet')
dat = dat.sample(.001)
dat.write.parquet('/home/shared/shishir/AurumOut/rawDat/MLM_for_pretraining_28M_1985_2020__unique_per6m_50pc_sample___10kdebug.parquet')


### not implementing unique per year 

In [ ]:
extractorMLM = BEHRTextraction()
dataout = extractorMLM.format_behrt(fullcodes, cohort, col_entry = 'enddate', unique_in_months=None)

In [ ]:
dataout.write.parquet('/home/shared/shishir/AurumOut/rawDat/MLM_for_pretraining_28M_1985_2020.parquet')


In [ ]:
dat = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/MLM_for_pretraining_28M_1985_2020.parquet')


# vocab creation

In [ ]:
fullcountsdiag = pd.read_parquet('/home/shared/shishir/AurumOut/Dicts/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021__CodeCounts.parquet')
fullcountsdiag =fullcountsdiag.rename(columns = {'count': 'val_counts','ICD':'target'})


fullcountsmed = pd.read_parquet('/home/shared/shishir/AurumOut/Dicts/meds_medcount_full.parquet').rename(columns = {'medcounts': 'val_counts'})
fullcountsmed = fullcountsmed[['target', 'val_counts']]

fullcountsmed = fullcountsmed.groupby('target').agg(sum)
fullcountsmed['target'] = fullcountsmed.index
fullcountsmed  =fullcountsmed.reset_index(drop=True)
fullcountsmed = fullcountsmed[['target', 'val_counts']]



In [ ]:
fullcounttotal = pd.concat((fullcountsmed, fullcountsdiag))
total =fullcounttotal.val_counts.sum()


In [ ]:

num_to_keep=25000
'code coverage:',(fullcounttotal[fullcounttotal.val_counts>num_to_keep].val_counts.sum()/total) , 'num codes kept:' , (len(fullcounttotal[fullcounttotal.val_counts>num_to_keep])/len(fullcounttotal))



In [ ]:
for num_to_keep in [1000*xx for xx in [10,20,30,40,50,100,150]]:
    print(num_to_keep, 'code coverage:',(fullcountsdiag[fullcountsdiag.val_counts>num_to_keep].val_counts.sum()/total) , 'num codes kept:' , (len(fullcountsdiag[fullcountsdiag.val_counts>num_to_keep])/len(fullcountsdiag))
)

In [ ]:
for num_to_keep in [1000*xx for xx in [10,20,30,40,50,100,150]]:
    print(num_to_keep, 'code coverage:',(fullcountsmed[fullcountsmed.val_counts>num_to_keep].val_counts.sum()/total) , 'num codes kept:' , (len(fullcountsmed[fullcountsmed.val_counts>num_to_keep])/len(fullcountsmed))
)

In [ ]:
for num_to_keep in [1000*xx for xx in [1, 10,20,30,40,50,100,150]]:
    print(num_to_keep, 'code coverage:',(fullcounttotal[fullcounttotal.val_counts>num_to_keep].val_counts.sum()/total) , 'num codes kept:' , (len(fullcounttotal[fullcounttotal.val_counts>num_to_keep])/len(fullcounttotal))
)

In [ ]:
ddcounts = {x:y for x, y in zip (codes.target.values, codes.val_counts.values) }

In [ ]:
vocabD = vocabCreate(ddcounts, 25000)
save_obj(vocabD, 'tempdict')